In [12]:
import pandas as pd
df = pd.read_csv('./pseudo/twitch.tsv',sep='\t')
df.head()

,text,none,curse
0,ㅋㅋ,0.993,0.007
1,a를 더 자주누르면되는거 아닌가? ㅋㅋㅋㅋ,0.992,0.008
2,그 누구지 땅우양은 그냥 사거릴 외우라던대,0.992,0.008
3,선생님 두오는 몬가요?,0.992,0.008
4,뭐 자신만의 카이팅이 있으니까..,0.992,0.008


In [13]:
df2 = pd.read_csv('labeled.csv')
df2.head()

,text,label
0,좌배 까는건 ㅇㅂ,0
1,집에 롱 패딩만 세 개다 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [14]:
df = list(df['text'])
df2 = list(df2['text'])

In [ ]:
# !pip install soynlp emoji

In [23]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = list({y for x in emoji.UNICODE_EMOJI.values() for y in x.keys()})
emojis = ''.join(emojis)
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [27]:
with open('./pseudo/corpus.txt', 'a', encoding='utf-8') as f:
    for text in df:
        f.write(clean(text)+'\n')
    for text in df2:
        f.write(clean(text)+'\n')

In [28]:
from tokenizers import BertWordPieceTokenizer

# load sentences
tokenizer = BertWordPieceTokenizer(
    vocab=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
    wordpieces_prefix="##",
)

limit_alphabet = 100000
vocab_size = 500000

tokenizer.train(
    files='./pseudo/corpus.txt',
    vocab_size=vocab_size,
    limit_alphabet=limit_alphabet,
    min_frequency=2,
    show_progress=True,
)

tokenizer.save("./pretrained_tokenizer/tokenizer.json", True)  # save tokenizer.json, pretty=True로 두시면 json 형식이 보기좋게 저장됩니다
tokenizer.save_model('./pretrained_tokenizer/')  # save vocab.txt

['./pretrained_tokenizer/vocab.txt']